<a href="https://colab.research.google.com/github/gyuchangShim/MJU_AI/blob/MASK-RCNN/Mask_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
!pip install pyyaml --upgrade
# PyTorch와 관련 패키지 설치
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Detectron2 설치
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.0/index.html

In [34]:
import tensorflow as tf
import os
import random
import cv2
import torch
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader

In [30]:
print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tf.test.is_gpu_available()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1516622366415340083
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18446225502897783598
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [31]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
dataset_dir = "/content/gdrive/MyDrive/pole_dataset/"
train_images_path = os.path.join(dataset_dir, "images")
train_annotations_path = os.path.join(dataset_dir, "annotations", "instances_default.json")
#val_images_path = os.path.join(dataset_dir, "val", "images")
#val_annotations_path = os.path.join(dataset_dir, "val", "annotations", "instances_val.json")

DatasetCatalog.remove("pole_train")
MetadataCatalog.remove("pole_train")

# COCO 데이터셋 등록
register_coco_instances("pole_train", {}, train_annotations_path, train_images_path)
#register_coco_instances("pole_val", {}, val_annotations_path, val_images_path)

# 데이터셋 메타데이터 로드
pole_metadata = MetadataCatalog.get("pole_train")

In [35]:
# 모델 구성 및 훈련
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("output", exist_ok=True)
            output_folder = "output"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

def train_model():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("pole_train",)
    cfg.DATASETS.TEST = ()
#    cfg.DATASETS.TEST = (("pole_val",))
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 5  # 데이터셋에 맞게 조정
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # 클래스 수 (배경 제외)

# 학습 진행
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
losses = []
accuracies = []

for epoch in range(cfg.SOLVER.MAX_ITER):
    trainer.train()
    # 각 epoch마다 훈련 데이터셋에 대한 손실(loss) 계산
    losses.append(trainer.storage.history("total_loss").latest())
    # 검증 데이터셋이 있다면 정확도(accuracy) 계산
    # 검증 데이터셋에 대한 정확도 계산 방법은 사용하는 데이터셋 및 평가 방법에 따라 다를 수 있습니다.
    # accuracies.append(calculate_accuracy(validation_data))

# Loss 그래프 생성
plt.plot(range(1, cfg.SOLVER.MAX_ITER + 1), losses, label='Training loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'cfg' is not defined